<a href="https://colab.research.google.com/github/unknown-jun/NLP_study/blob/main/Self_Implement/DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch DataLoader
Pytorch의 `Dataset`클래스는 데이터를 **전처리**하고 dictionary 또는 list 타입으로 변경할 수 있다.  
`DataLoader` 클래스는 데이터 **1. 셔플 2. 배치화 3. 멀티 프로세스** 기능을 제공한다.

## Dataset
- 모든 custom dataset 클래스는 `Dataset()` 클래스를 상속 받아야 함
- `__getitem__()`와 `__len__()` 메소드는 반드시 오버라이딩해야 함.
- `__getitem__()`는 key를 통해서 indexing 할 수 있게 하는 자료형을 만들어 주는 special method
- `DataLoader` 클래스가 배치를 만들 때 `Dataset` 인스턴스의 `__getitem__()` 메소드를 사용해 데이터에 접근함
- 해당 Dataset 클래스는 string sequence 데이터를 **tokenize & tensorize**한다.

In [ ]:
import numpy as np

class MyDataset(Dataset):
  def __init__(self):
    x = np.random.rand(1000,3)
    self.x = [x[i].tolist() for i in range(1000)]
    y = np.random.randint(low=0, high=2, size=(1000,))
    self.y = [y[i] for i in range(1000)]

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

def collate_fn(batch):
  print(f"batch: {batch}")
  data_list, label_list = [],[]
  for _data, _label in batch:
    data_list.append(_data)
    label_list.append(_label)
  return torch.Tensor(data_list), torch.Tensor(label_list)

if __name__ == '__main__':
  dataset = MyDataset()
  print(len(dataset))
  print(dataset[-1])

  dataloader = DataLoader(dataset, batch_size=3, shuffle=False, collate_fn=collate_fn)
  for data, label in dataloader:
    print(type(data))
    print(data)
    print(type(label))
    print(label)
    break

1000
([0.581929502462736, 0.6157731528496572, 0.9507509145072667], 0)
batch: [([0.7772657187523754, 0.917082529878135, 0.5733755440935383], 0), ([0.7539771442117729, 0.47075378909765564, 0.8587395524244688], 1), ([0.8375300060120578, 0.5927706918002297, 0.9887690929041181], 0)]
<class 'torch.Tensor'>
tensor([[0.7773, 0.9171, 0.5734],
        [0.7540, 0.4708, 0.8587],
        [0.8375, 0.5928, 0.9888]])
<class 'torch.Tensor'>
tensor([0., 1., 0.])


# Week2_3 Assignment

## [BASIC](#Basic) 
- **Custom Dataset 클래스를 구현**할 수 있다.
- train_dataset, valid_dataset으로 데이터셋을 나눌 수 있다.


## [CHALLENGE](#Challenge)
- **dynamic padding**을 만드는 `collate_fn`을 구현할 수 있다. 
- `DataLoader` 클래스를 사용해 **train_dataloaer와 valid_dataloader**를 만들 수 있다.


## [ADVANCED](#Advanced)
- 기존 코드의 data_iterator를 **data_loader로 대체**해 학습을 실행할 수 있다.

### Reference
-[DataLoader pytorch official document](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)   
-[collate_fn 설명 영문 블로그](https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/)   
-[dynamic padding 설명 영문 블로그](https://mccormickml.com/2020/07/29/smart-batching-tutorial/#dynamic-padding)   


In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

In [ ]:
# seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


## Basic

### 데이터 로드 및 결측치 제거 (복습)
- 해당 링크에서 `sample_df` 데이터 프레임을 로드하자
  - df의 개수는 10,000개이다.

In [ ]:
!wget https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/e56006adfac42f8a2975db0ebbe60eacbe1c6b11/data/sample_df.csv

--2022-07-10 06:21:32--  https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/e56006adfac42f8a2975db0ebbe60eacbe1c6b11/data/sample_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971625 (949K) [text/plain]
Saving to: ‘sample_df.csv’

sample_df.csv       100%[===================>] 948.85K  --.-KB/s    in 0.05s   

2022-07-10 06:21:32 (20.1 MB/s) - ‘sample_df.csv’ saved [971625/971625]



In [ ]:
sample_df = pd.read_csv('sample_df.csv')
sample_df.head()

,id,document,label
0,8525343,나 이거 더빙을 누가하는지 모르고 봤는데 왠지 더빙이 구리더라...더빙이 너무 별로였음.,0
1,4572888,현암이 소지섭이었으면 좋았겠는데..스토리각색도 좀 깔끔하게...,0
2,8504845,"ㅎㅎㅎ 대단하네 ㅜ,.ㅡ",0
3,5003367,이거보고 돈날린 기억이...........,0
4,3015049,한국영화 어쩌다 이지경까지 ㅠㅠ,0


In [ ]:
print(f"Shape: {sample_df.shape}\nLabel : {sample_df.label.value_counts()}")

Shape: (10000, 3)
Label : 0    5000
1    5000
Name: label, dtype: int64


### `CustomDataset `클래스 구현
- 클래스 정의
  - 생성자 입력 매개변수 
    - `input_data` : 리뷰 텍스트 리스트
    - `target_data` : 레이블 (0 또는 1) list
  - 생성자에서 생성할 변수
    - `X` : `input_data`
    - `Y` : `target_data`
  - 메소드 
    - `__len__()`
      - 데이터 총 개수를 반환
    - `__getitem__()`
      - 해당 인덱스의 (input_data, target_data) 튜플을 반환
  - 주의 사항
    - `torch.utils.data.Dataset()` 클래스를 부모 클래스로 상속받아 구현한다.  

In [ ]:
!pip install transformers

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
from transformers import BertTokenizer

In [ ]:
class CustomDataset(Dataset):
  """
  - input_data: list of string
  - target_data: list of int
  """

  def __init__(self, input_data:list, target_data:list) -> None:
      self.X = input_data
      self.Y = target_data

  def __len__(self):
      return len(self.X)

  def __getitem__(self, index):
      return self.X[index], self.Y[index]

In [ ]:
dataset = CustomDataset(sample_df['document'].to_list(), sample_df['label'].to_list())

In [ ]:
# map-stype dataset 클래스는 indexing이 가능함
dataset[100]

('감독이 촉나라 안티냐 유관장3형제는 조선족 50대아저씨 들이고 오나라는 카리스마 미끈외모 홍콩귀족이냐고..', 0)

In [ ]:
# 데이터 셋 총 개수 확인 가능
len(dataset)

10000

###  `torch.utils.data.random_split` 함수를 사용해 데이터셋을 train, valid로 분리
- 데이터 비율은 `train : valid = 9 : 1`으로 분리하라

In [ ]:
n_train = int(10000 * 0.9)
n_valid = int(10000 * 0.1)

In [87]:
train_dataset, valid_dataset = random_split(dataset, [n_train, n_valid])

In [88]:
len(train_dataset)

9000

In [89]:
len(valid_dataset)

1000

## Challenge

### dynamic padding을 구현하는  `custom_collate_fn` 함수 구현 
- batch (`List[Tuple(input, target)]`)를 입력받아 토크나이즈한 후 텐서 형태로 변형해 반환 ( `Tuple(Tensor(tokenized_input), Tensor(target))`)하는 `collate_fn()` 함수를 구현하라. 
- 함수 정의
  - 입력 매개변수
    - `batch` : (input(string), target(int)) 튜플을 담고 있는 리스트.  만약 `batch_size`가 32라면 리스트는 32개의 튜플을 갖고 있다. 
  - 조건
    - input
      - [BERT Tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer) 클래스의 `__call__()` 메소드 사용 방법을 읽고, `__call__()` 파라미터를 조정해 dynamic padding을 구현한다.
      - 토크나이즈할 때 한 배치내 인풋들의 토큰 개수는 모두 동일할 수 있도록하라. 이때, 가장 긴 토큰을 가지고 있는 인풋을 기준으로 토큰 개수를 맞춘다. 즉, 토큰 개수가 모자란 인풋은 `[PAD]` 토큰을 추가한다. (이를 **dynamic padding**이라고 한다.) 
      - 토크나이저에서 반환된 값은 Tensor 타입이어야 한다. 
    - target
      - target은 Tensor 타입으로 변형한다.
  - 반환값
    - (tensorized_input, tensorized_label) 튜플

In [90]:
from transformers import BertTokenizer, BertModel

In [91]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

In [92]:
def custom_collate_fn(batch):
  """
  - batch: list of tuples (input_data(string), target_data(int))
  
  한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
  이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
  토큰 개수는 배치 내 가장 긴 문장으로 해야함.
  또한 최대 길이를 넘는 문장은 최대 길이 이후의 토큰을 제거하도록 해야 함
  토크나이즈된 결과 값은 텐서 형태로 반환하도록 해야 함
  
  한 배치 내 레이블(target)은 텐서화 함.
  
  (input, target) 튜플 형태를 반환.
  """
  global tokenizer_bert
  
  input_list, target_list = [ ], [ ]

  for _input, _label in batch:
    input_list.append(_input)
    target_list.append(_label)
  
  tensorized_input = tokenizer_bert(input_list,
                                    add_special_tokens=True,
                                    truncation=True,
                                    padding='longest',
                                    max_length=512,
                                    return_tensors='pt')
  
  tensorized_label = torch.tensor(target_list)
  return tensorized_input, tensorized_label
    

### 위에서 구현한 `custom_collate_fn`함수를 적용해 DataLoader 인스턴스 생성
- `train_dataloader`
    - batch_size = 32
    - collate_fn = 위에서 구현한 함수
    - sampler = `RandomSampler()`
        - `train_dataset`의 학습 데이터를 셔플링 함
- `valid_dataloader`
    - batch_size = 64
    - collate_fn = 위에서 구현한 함수
    - sampler = `SequentialSampler()`
        - `valid_dataset`의 검증 데이터를 순차적으로 정렬함 (셔플X)

In [93]:
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=32, 
                              collate_fn= custom_collate_fn, 
                              sampler= RandomSampler(train_dataset))

valid_dataloader = DataLoader(dataset=valid_dataset,
                              batch_size=64, 
                              collate_fn= custom_collate_fn, 
                              sampler= SequentialSampler(valid_dataset))

In [94]:
# 배치마다 토큰 길이가 다른 것을 확인
for input_batch, target_batch in valid_dataloader:
  print(f"Batch input shape: {input_batch['input_ids'].shape}")
  print(f"Batch target shape: {target_batch.shape}")

Batch input shape: torch.Size([64, 78])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 77])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 75])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 75])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 84])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 83])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 75])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 79])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 98])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 91])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 74])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 81])
Batch target shape: torch.Size([64])
Batch input shape: torch.Size([64, 85])
Batch target shape: torch.Size([64])

## Advanced

### 어제(week2-2) 생성한 `train()` 함수의 입력값이었던 `data_iterator`를  `data_loader`로 변경

In [97]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

In [135]:
def train(model, data_loader):
  global loss_fct

  # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
  total_loss, batch_loss, batch_count = 0,0,0
  
  # model을 train 모드로 설정 & device 할당
  model.train()
  model.to(device)
  
  # data iterator를 돌면서 하나씩 학습
  for step, batch in enumerate(data_loader):
      batch_count+=1
      
      # tensor 연산 전, 각 tensor에 device 할당
      batch_input, batch_label = batch
      batch_input, batch_label = batch_input.to(device), batch_label.to(device)
      
      # batch마다 모델이 갖고 있는 기존 gradient를 초기화
      model.zero_grad()
      
      # forward
      logits = model(**batch_input)
      
      # loss
      loss = loss_fct(logits, batch_label)
      batch_loss += loss.item()
      total_loss += loss.item()
      
      # backward -> 파라미터의 미분(gradient)를 자동으로 계산
      loss.backward()
      
      # optimizer 업데이트
      optimizer.step()
      
      # 배치 10개씩 처리할 때마다 평균 loss를 출력
      if (step % 10 == 0 and step != 0):
          print(f"Step : {step}, Avg Loss : {batch_loss / batch_count:.4f}")
          
          # 변수 초기화 
          batch_loss, batch_count = 0,0
  
  print(f"Mean Loss : {total_loss/(step+1):.4f}")
  print("Train Finished")

### 지금까지 구현한 함수와 클래스를 모두 불러와 `train()` 함수를 실행하자
- fine-tuning 모델 클래스 (`CustomClassifier`)
    - hidden_size = 768
    - n_label = 2
- 데이터 이터레이터 함수 (`data_iterator`)
    - batch_size = 32
- loss 
    - `CrossEntropyLoss()`
- optimizer
    - `AdamW()`
    - lr = 2e-5


In [136]:
# Week2-2에서 구현한 클래스와 동일

class CustomClassifier(nn.Module):

  def __init__(self, hidden_size: int, n_label: int):
    super(CustomClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('klue/bert-base')
    self.hidden_size = hidden_size
    self.n_label = n_label

    dropout_rate = 0.1
    linear_layer_hidden_size = 32

    self.classifier=nn.Sequential(nn.Linear(self.hidden_size, linear_layer_hidden_size),
                                  nn.ReLU(),
                                  nn.Dropout(dropout_rate),
                                  nn.Linear(linear_layer_hidden_size, n_label))

  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

    output = self.bert(
        input_ids,
        attention_mask = attention_mask,
        token_type_ids = token_type_ids
    )

    cls_token_last_hidden_states = output[0][:,0,:]
    logits = self.classifier(cls_token_last_hidden_states)

    return logits

In [137]:
# 모델
model = CustomClassifier(hidden_size = 768, n_label = 2)

# 데이터로더
batch_size = 32
train_dataloader = DataLoader(
                        train_dataset,
                        batch_size =batch_size,
                        sampler = RandomSampler(train_dataset),
                        collate_fn = custom_collate_fn
)

# 로스 및 옵티마이저
loss_fct = CrossEntropyLoss()
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8
)

# 학습 시작
train(model, train_dataloader)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step : 10, Avg Loss : 0.6834
Step : 20, Avg Loss : 0.6114
Step : 30, Avg Loss : 0.4797
Step : 40, Avg Loss : 0.4812
Step : 50, Avg Loss : 0.4546
Step : 60, Avg Loss : 0.4958
Step : 70, Avg Loss : 0.4277
Step : 80, Avg Loss : 0.3920
Step : 90, Avg Loss : 0.3633
Step : 100, Avg Loss : 0.3629
Step : 110, Avg Loss : 0.3696
Step : 120, Avg Loss : 0.3754
Step : 130, Avg Loss : 0.3615
Step : 140, Avg Loss : 0.3697
Step : 150, Avg Loss : 0.3806
Step : 160, Avg Loss : 0.3016
Step : 170, Avg Loss : 0.4219
Step : 180, Avg Loss : 0.3340
Step : 190, Avg Loss : 0.3682
Step : 200, Avg Loss : 0.3771
Step : 210, Avg Loss : 0.3386
Step : 220, Avg Loss : 0.3556
Step : 230, Avg Loss : 0.3157
Step : 240, Avg Loss : 0.3029
Step : 250, Avg Loss : 0.3350
Step : 260, Avg Loss : 0.3104
Step : 270, Avg Loss : 0.2685
Step : 280, Avg Loss : 0.3376
Mean Loss : 0.3922
Train Finished
